# Configuration de Cuda dans Google Colab

In [ ]:
!nvcc -V

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

## On vérifie que l'on est bien connecté au GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

## Chargement du plugin nvcc permettant de compiler/executer les programmes Cuda

In [ ]:
%load_ext nvcc_plugin

## Un makefile est déjà à votre disposition pour compiler les programme du TP


### 1. Executez la cellule du Makefile


In [ ]:
%%writefile Makefile
# Change the example variable to build a different source module (e.g. EXAMPLE=exercice01)
EXAMPLE=program

# Makefile variables 
# Add extra targets to OBJ with space separator e.g. If there is as source file random.c then add random.o to OBJ)
# Add any additional dependancies (header files) to DEPS. e.g. if there is aheader file random.h required by your source modules then add this to DEPS.
CC=gcc
CFLAGS= -O3 -Wextra -fopenmp
NVCC=nvcc
NVCC_FLAGS= -gencode arch=compute_75,code=sm_75
OBJ=$(EXAMPLE).o
DEPS=

# Build rule for object files ($@ is left hand side of rule, $< is first item from the right hand side of rule)
%.o : %.cu $(DEPS)
	$(NVCC) -c -o $@ $< $(NVCC_FLAGS) $(addprefix -Xcompiler ,$(CCFLAGS))

# Make example ($^ is all items from right hand side of the rule)
$(EXAMPLE) : $(OBJ)
	$(NVCC) -o $@ $^ $(NVCC_FLAGS) $(addprefix -Xcompiler ,$(CCFLAGS))

# PHONY prevents make from doing something with a filename called clean
.PHONY : clean
clean:
	rm -rf $(EXAMPLE) $(OBJ)

### 2. Executez la cellule program.cu





In [ ]:
%%writefile program.cu
#include <stdlib.h>
#include <stdio.h>
#include <math.h>

int main(void){
  printf("Hello World !\n");
  return 0;
}

### 3. Compilez et lancer le programme en executant les 2 cellules suivantes

In [ ]:
!make

In [ ]:
!./program

# TP 1

## 1. Fonctions utilitaires

utils.h est un header contenant des fonctions utilitaires qui seront utilisés par nos programmes

In [ ]:
%%writefile utils.h
#ifndef __UTILS_H__
#define __UTILS_H__
#include <stdio.h>

static void HandleError( cudaError_t err,
                         const char *file,
                         int line ) {
    if (err != cudaSuccess) {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}
#define HANDLE_ERROR( err ) (HandleError( err, __FILE__, __LINE__ ))


#define HANDLE_NULL( a ) {if (a == NULL) { \
                            printf( "Host memory failed in %s at line %d\n", \
                                    __FILE__, __LINE__ ); \
                            exit( EXIT_FAILURE );}}

template< typename T >
void swap( T& a, T& b ) {
    T t = a;
    a = b;
    b = t;
}


void* big_random_block( int size ) {
    unsigned char *data = (unsigned char*)malloc( size );
    HANDLE_NULL( data );
    for (int i=0; i<size; i++)
        data[i] = rand();

    return data;
}

int* big_random_block_int( int size ) {
    int *data = (int*)malloc( size * sizeof(int) );
    HANDLE_NULL( data );
    for (int i=0; i<size; i++)
        data[i] = rand();

    return data;
}


// a place for common kernels - starts here

__device__ unsigned char value( float n1, float n2, int hue ) {
    if (hue > 360)      hue -= 360;
    else if (hue < 0)   hue += 360;

    if (hue < 60)
        return (unsigned char)(255 * (n1 + (n2-n1)*hue/60));
    if (hue < 180)
        return (unsigned char)(255 * n2);
    if (hue < 240)
        return (unsigned char)(255 * (n1 + (n2-n1)*(240-hue)/60));
    return (unsigned char)(255 * n1);
}

__global__ void float_to_color( unsigned char *optr,
                              const float *outSrc ) {
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;

    float l = outSrc[offset];
    float s = 1;
    int h = (180 + (int)(360.0f * outSrc[offset])) % 360;
    float m1, m2;

    if (l <= 0.5f)
        m2 = l * (1 + s);
    else
        m2 = l + s - l * s;
    m1 = 2 * l - m2;

    optr[offset*4 + 0] = value( m1, m2, h+120 );
    optr[offset*4 + 1] = value( m1, m2, h );
    optr[offset*4 + 2] = value( m1, m2, h -120 );
    optr[offset*4 + 3] = 255;
}

__global__ void float_to_color( uchar4 *optr,
                              const float *outSrc ) {
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;

    float l = outSrc[offset];
    float s = 1;
    int h = (180 + (int)(360.0f * outSrc[offset])) % 360;
    float m1, m2;

    if (l <= 0.5f)
        m2 = l * (1 + s);
    else
        m2 = l + s - l * s;
    m1 = 2 * l - m2;

    optr[offset].x = value( m1, m2, h+120 );
    optr[offset].y = value( m1, m2, h );
    optr[offset].z = value( m1, m2, h -120 );
    optr[offset].w = 255;
}


#if _WIN32
    //Windows threads.
    #include <windows.h>

    typedef HANDLE CUTThread;
    typedef unsigned (WINAPI *CUT_THREADROUTINE)(void *);

    #define CUT_THREADPROC unsigned WINAPI
    #define  CUT_THREADEND return 0

#else
    //POSIX threads.
    #include <pthread.h>

    typedef pthread_t CUTThread;
    typedef void *(*CUT_THREADROUTINE)(void *);

    #define CUT_THREADPROC void
    #define  CUT_THREADEND
#endif

//Create thread.
CUTThread start_thread( CUT_THREADROUTINE, void *data );

//Wait for thread to finish.
void end_thread( CUTThread thread );

//Destroy thread.
void destroy_thread( CUTThread thread );

//Wait for multiple threads.
void wait_for_threads( const CUTThread *threads, int num );

#if _WIN32
    //Create thread
    CUTThread start_thread(CUT_THREADROUTINE func, void *data){
        return CreateThread(NULL, 0, (LPTHREAD_START_ROUTINE)func, data, 0, NULL);
    }

    //Wait for thread to finish
    void end_thread(CUTThread thread){
        WaitForSingleObject(thread, INFINITE);
        CloseHandle(thread);
    }

    //Destroy thread
    void destroy_thread( CUTThread thread ){
        TerminateThread(thread, 0);
        CloseHandle(thread);
    }

    //Wait for multiple threads
    void wait_for_threads(const CUTThread * threads, int num){
        WaitForMultipleObjects(num, threads, true, INFINITE);

        for(int i = 0; i < num; i++)
            CloseHandle(threads[i]);
    }

#else
    //Create thread
    CUTThread start_thread(CUT_THREADROUTINE func, void * data){
        pthread_t thread;
        pthread_create(&thread, NULL, func, data);
        return thread;
    }

    //Wait for thread to finish
    void end_thread(CUTThread thread){
        pthread_join(thread, NULL);
    }

    //Destroy thread
    void destroy_thread( CUTThread thread ){
        pthread_cancel(thread);
    }

    //Wait for multiple threads
    void wait_for_threads(const CUTThread * threads, int num){
        for(int i = 0; i < num; i++)
            end_thread( threads[i] );
    }

#endif




#endif  // __UTILS_H__

Le code suivant vous permet d'afficher certaines propriétés de la carte graphiqe qui seront expliqué en cours.


In [ ]:
%%writefile program.cu
#include <stdio.h>
#include "utils.h"

int main(void){
  cudaDeviceProp prop;

  int count;
  HANDLE_ERROR(cudaGetDeviceCount(&count));

  for (int i=0; i< count; i++) {
    HANDLE_ERROR(cudaGetDeviceProperties(&prop, i));
    printf("Name: %s\n", prop.name);
    printf("Compute capability: %d.%d\n", prop.major, prop.minor);
    printf("Clock rate: %d\n", prop.clockRate);
    printf("Device copy overlap: ");
    if (prop.deviceOverlap)
      printf("Enabled\n");
    else
      printf("Disabled\n");
    printf("Kernel execution timeout : ");
    if (prop.kernelExecTimeoutEnabled)
      printf("Enabled\n");
    else
      printf("Disabled\n");

      printf(" --- Memory Information for device %d ---\n", i);
      printf("Total global mem: %ld\n", prop.totalGlobalMem);
      printf("Total constant mem: %ld\n", prop.totalConstMem);
      printf("Max mem pitch: %ld\n", prop.memPitch);
      printf("Texture Alignment: %ld\n", prop.textureAlignment);

      printf(" --- MP Information for device %d ---\n", i);
      printf("multiprocessor count : %d\n", prop.multiProcessorCount);
      printf("shared mem per mp: %ld\n", prop.sharedMemPerBlock);
      printf("Registers per mp: %d\n", prop.regsPerBlock);
      printf("Threads in warp: %d\n", prop.warpSize);
      printf("Max threads per block: %d\n", prop.maxThreadsPerBlock);
      printf("Max thread dimensions: (%d, %d, %d)\n", prop.maxThreadsDim[0], prop.maxThreadsDim[1], prop.maxThreadsDim[2]);
      printf("Max grid dimensions: (%d, %d, %d)\n", prop.maxGridSize[0], prop.maxGridSize[1], prop.maxGridSize[2]);
      printf("\n");

  }
  return 0;
}

In [ ]:
!make

In [ ]:
!./program

## 2. Votre premier programme CUDA

In [ ]:
%%writefile program.cu
#include <stdio.h>
#include "utils.h"

__global__ void add(int a, int b, int *c) {
  *c = a + b;
}

int main(void){
  int c;
  int *dev_c;
  HANDLE_ERROR(cudaMalloc((void**)&dev_c, sizeof(int)));
  add<<<1,1>>>(2, 7, dev_c);
  HANDLE_ERROR(cudaMemcpy(&c, dev_c, sizeof(int), cudaMemcpyDeviceToHost));
  printf("2 + 7 = %d\n", c);
  cudaFree(dev_c);
  return 0;
}

### Program.cu

Si vous avez des bases en programmation C, vous devriez noter quelques différences entre un programme classique et le program.cu :


*   La fonction void add(int a, int b, int \*c) est préfixé de \_\_global\_\_ \\

  Cela veut dire que la fonction est un **kernel**, une fonction s'exécutant sur la carte graphique. Le compilateur comprend que la fonction s'exècute sur la GPU grâce au préfixe \_\_global\_\_.
*   L'instruction cudaMalloc((void\*\*)&dev_c, sizeof(int)) \\

  Vous avez du voir en C que pour allouer de la mémoire à un pointeur, on utilise la fonction **malloc**. **cudaMalloc** permet d'allouer de la mémoire à un pointeur directement sur la RAM de la carte graphique.
*   cudaFree(dev_c) \\

  Pareil que **free** en C pour libérer la mémoire d'un pointeur, mais libère la mémoire GPU.
*   add<<<1,1>>>(2, 7, dev_c) \\

  C'est comme cela que l'on lance un kernel (une fonction préfixé de \_\_global\_\_ qui s'exécute sur la carte graphique) en Cuda. Le premier 1 correspond au nombre de blocs que l'on veut utiliser, le deuxième au nombre de threads. Ne vous inquiétez pas si vous ne comprenez pas bien ces termes, nous reviendrons dessus plus tard. 

  Vous pouvez le voir comme cela : add<<<nb_blocs, nb_threads>>>(2, 7, dev_c), avec nb_blocs égale à 1 et nb_threads égale à 1.

*   cudaMemcpy(&c, dev_c, sizeof(int), cudaMemcpyDeviceToHost) \\

  Une fois les instructions du kernel ayant terminé de s'exécuter sur le GPU, le résultat est stocké dans le pointeur dev_c, c'est à dire dans la mémoire RAM de la carte graphique. Afin de pouvoir utiliser ce résultat dans le **main** qui est utilisé par le CPU, il nous faut copier le contenu de dev_c dans la mémoire RAM de l'ordinateur, qui est utilisé par le CPU. Cela se fait grâce à l'instruction **cudaMemcpy**, qui dans cet exemple copie le contenu de dev_c dans le pointeur c.

* HANDLE_ERROR() \\

  Il n'est pas évident de débuguer un programme Cuda, nous verrons plus tard pourquoi. Chaque instruction Cuda est appelé à l'intérieur de la fonction HANDLE_ERROR() pour que l'on puisse savoir laquelle a produit un erreur.

Compilez et exécutez le programme.


In [ ]:
!make

In [ ]:
!./program

## 3. Addition de vecteurs

Le but de cette exercice est de se familiariser avec les bases de cuda en programmant une addition de vecteurs sur GPU. Le programme suivant est un programme classique en C qui additionne deux vecteurs et stocke le résultat dans le vecteur **c**.

In [ ]:
%%writefile program.cu
#include <stdio.h>
#include <time.h>
#include "utils.h"

#define N 10

void add(int *a, int *b, int *c){
  int tid = 0; // This is the CPU 0, so we start at 0
  while(tid < N){
    c[tid] = a[tid] + b[tid];
    tid += 1; // We have one CPU, so we increment by one
  }
}

int main(void){
  int a[N], b[N], c[N];
  
  // fill the arrays 'a' and 'b' on the CPU
  for (int i=0; i<N; i++){
    a[i] = -i;
    b[i] = i * i;
  }

  add(a, b, c);

  //display the results
  for(int i=0; i<N; i++){
    printf("%d + %d = %d\n", a[i], b[i], c[i]);
  }

  return 0;
}

In [ ]:
!make

In [ ]:
!./program

### 3.1 Addition de vecteurs par blocs de threads

Le but de cet exercice est d'additioner les deux vecteurs **a** et **b** sur GPU en utilisant N blocs de 1 threads.

Le kernel pour cet exercice vous est donné : 

```c
__global__ void add(int *a, int *b, int *c){
  int tid = blockIdx.x;
  if(tid < N)
    c[tid] = a[tid] + b[tid];
}
```
Vous remarquerez qu'il n'y a pas de boucle dans cette fonction. En effet, chaque bloc contient 1 unique threads, ce qui nous donne N threads (car N blocs). Chaque bloc se contente d'additioner un et un seul élément des vecteur de taille N. Les blocs s'executent en parallèle.

Pour récupérer l'indice d'un bloc, on récupère la variable x de la structure CUDA blockIdx. 

En vous aidant de votre premier programme CUDA, complétez le code suivant :

1.   Allouez la mémoire des pointeurs \*dev_a, \*dev_b, \*dev_c.
2.   Copiez le contenu du pointeur **a** dans **dev_a** et de **b** dans **dev_b**
3. Lancez le kernel avec N blocs et 1 thread
4. Copiez le contenu du pointeur **dev_c** dans **c**
5. Libérez la mémoire des pointeurs.






In [ ]:
%%writefile program.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "utils.h"


#define N 10

__global__ void add(int *a, int *b, int *c){
  int tid = blockIdx.x;
  if(tid < N)
    c[tid] = a[tid] + b[tid];
}

int main(void){
  int a[N], b[N], c[N];
  int *dev_a, *dev_b, *dev_c;

  // 1. Allocation de mémoire
  // ECRIVEZ LE CODE ICI

  // fill the arrays 'a' and 'b' on the CPU
  for (int i=0; i<N; i++){
    a[i] = -i;
    b[i] = i * i;
  }

  // 2. Copie de mémoire CPU(Host) vers GPU(Device)
  // ECRIVEZ LE CODE ICI
  
  // 3. Lancement du kernel
  // ECRIVEZ LE CODE ICI

  // 4. Copie de mémoire GPU vers CPU
  // ECRIVEZ LE CODE ICI
  
  // Display the results
  for(int i=0; i<N; i++){
    printf("%d + %d = %d\n", a[i], b[i], c[i]);
  }

  // 4. Libéré la mémoire
  // ECRIVEZ LE CODE ICI
  return 0;
}

In [ ]:
!make

In [ ]:
!./program

### 3.2 Addition de vecteurs par threads d'un seul bloc
Le but de cet exercice est le même que précédement sauf que à la place d'avoir N blocs de 1 threads, on utilise 1 bloc de N threads

Le kernel ne vous ai pas donné cette fois.

Complétez le code suivant :

1. Complétez le kernel pour avoir une indexation par thread et non par bloc
2.   Allouez la mémoire des pointeurs \*dev_a, \*dev_b, \*dev_c.
3.   Copiez le contenu du pointeur **a** dans **dev_a** et de **b** dans **dev_b**
4. Lancez le kernel avec N blocs et 1 thread
5. Copiez le contenu du pointeur **dev_c** dans **c**
6. Libérez la mémoire des pointeurs.


In [ ]:
%%writefile program.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "utils.h"

#define N 10

__global__ void add(int *a, int *b, int *c){
  // 1. Addition et indexation par thread
  // ECRIVEZ LE CODE ICI
}

int main(void){
  int a[N], b[N], c[N];
  int *dev_a, *dev_b, *dev_c;

  // 2. Allocation de mémoire
  // ECRIVEZ LE CODE ICI

  // fill the arrays 'a' and 'b' on the CPU
  for (int i=0; i<N; i++){
    a[i] = -i;
    b[i] = i * i;
  }

  // 3. Copie de mémoire CPU(Host) vers GPU(Device)
  // ECRIVEZ LE CODE ICI
  
  // 4. Lancement du kernel
  // ECRIVEZ LE CODE ICI

  // 5. Copie de mémoire GPU vers CPU
  // ECRIVEZ LE CODE ICI
  
  // Display the results
  for(int i=0; i<N; i++){
    printf("%d + %d = %d\n", a[i], b[i], c[i]);
  }

  // 6. Libéré la mémoire
  // ECRIVEZ LE CODE ICI
  return 0;
}

In [ ]:
!make

In [ ]:
!./program

### 3.3 Multiplication vectorielle par blocs de threads
Nous avons maintenant 256 threads par bloc. Le but de cet exercice est de trouver le nombre de blocs optimal pour réaliser un multiplication vectorielle sur GPU.

Complétez le code suivant :

1. Trouvez le nombre de bloc optimal à utiliser. 
2. Trouvez l'index global des threads

In [ ]:
%%writefile program.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define N 10000

__global__ void multiply(float* a, float* b, float* c, int n) {
  int id = // 2. Trouver l'index global des threads
  if (id < n) {
    c[id] = a[id] * b[id];
  }
}

int main() {
  float *a, *b, *c;
  cudaMallocManaged(&a, N * sizeof(float));
  cudaMallocManaged(&b, N * sizeof(float));
  cudaMallocManaged(&c, N * sizeof(float));

  for (int i = 0; i < N; i++) {
    a[i] = rand() / (float)RAND_MAX;
    b[i] = rand() / (float)RAND_MAX;
  }

  int nb_threads = 256;
  int nb_blocks = // 1. Trouvez le nombre de bloc optimal.

  multiply<<<nb_blocks, nb_threads>>>(a, b, c, N);
  cudaDeviceSynchronize();

  float error = 0.0;
  for (int i = 0; i < N; i++) {
    error += fabs(c[i] - a[i] * b[i]);
  }
  printf("Error: %f\n", error);

  cudaFree(a);
  cudaFree(b);
  cudaFree(c);

  return 0;
}


In [ ]:
!make

In [ ]:
!./program